# Baseline Result on SST2 Sentiment Analysis using RoBerTa

In [2]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import numpy as np 
import matplotlib.pyplot as plt
import nltk
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, roc_curve, auc
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import os
from sklearn.metrics import balanced_accuracy_score
!pip install autocorrect
import pickle
from autocorrect import Speller
import re
import string
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
!pip install emoji
import emoji


     |████████████████████████████████| 622 kB 14.4 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=02fb24d477eed2a2c897a28d61cc5b22e80917ef597876006cbc7a42dc306236
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect
     |████████████████████████████████| 175 kB 13.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=8e6dafc94714591fec74a387404d882f40728ccd4cbc440752bc55269997c2f5
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [5]:
## the HuggingFace libraries are not already installed in Colab
!pip install transformers==4.17 # this will also install tokenizers 
!pip install datasets

     |████████████████████████████████| 3.8 MB 15.3 MB/s 
     |████████████████████████████████| 6.6 MB 62.6 MB/s 
     |████████████████████████████████| 596 kB 69.1 MB/s 
     |████████████████████████████████| 77 kB 9.6 MB/s 
     |████████████████████████████████| 895 kB 58.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 14.3 MB/s 
     |████████████████████████████████| 212 kB 59.0 MB/s 
     |████████████████████████████████| 136 kB 57.3 MB/s 
     |████████████████████████████████| 1.1 MB 82.0 MB/s 
     |████████████████████████████████| 127 kB 53.8 MB/s 
     |████████████████████████████████| 271 kB 58.2 MB/s 
     |████████████████████████████████| 144 kB 93.5 MB/s 
     |████████████████████████████████| 94 kB 2.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [6]:
import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer, RobertaForSequenceClassification,RobertaModel, Trainer, TrainingArguments
from transformers import AdamW, get_linear_schedule_with_warmup
import datasets
print(transformers.__version__)

4.17.0


# 3. Sentiment Analysis using Pre-trained Roberta-base model

In our case: **classify tweets about airlines as positive (label: 1) / negative (label: 0)**.

Fine-tuning BERT for sequence classification: add a classifier on top of BERT.

The classifier is made of several fully-connected layers.

## Data

In [7]:
# The datasets library enables to load a dataset by just calling datasets.load_data("dataset_name")
data = datasets.load_dataset("SetFit/sst2")

Using custom data configuration SetFit--sst2-c66a8eb4897c1f8f


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/SetFit--sst2-c66a8eb4897c1f8f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# View data splits
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 6920
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1821
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 872
    })
})

In [8]:
data_train = datasets.Dataset.to_pandas(data["train"])
data_val = datasets.Dataset.to_pandas(data["validation"])
data_test = datasets.Dataset.to_pandas(data["test"])

In [ ]:
data_test.head()

,text,label,label_text
0,"no movement , no yuks , not much of anything .",0,negative
1,"a gob of drivel so sickly sweet , even the eag...",0,negative
2,"gangs of new york is an unapologetic mess , wh...",0,negative
3,"we never really feel involved with the story ,...",0,negative
4,this is one of polanski 's best films .,1,positive


In [ ]:
len(data["train"]["label"]) # the 1st data point is negative

6920

In [ ]:
# classes
np.unique(data["train"]["label"])

array([0, 1])

In [ ]:
# % positives in train/validation/test
pos_frac_train = 100 * np.mean([data["train"]["label"][i] for i in range(len(data["train"]["label"]))])
pos_frac_val = 100 * np.mean([data["validation"]["label"][i] for i in range(len(data["validation"]["label"]))])
pos_frac_test = 100 * np.mean([data["test"]["label"][i] for i in range(len(data["test"]["label"]))])
pos_frac_train, pos_frac_val, pos_frac_test

(52.16763005780347, 50.917431192660544, 49.9176276771005)

## Dataset class

In [9]:
class SST2Dataset():
## Dataset in pandas dataframe format##
    def __init__(self, tokenizer, df, max_length):
        super(SST2Dataset, self).__init__()

        self.tokenizer = tokenizer
        self.max_seq_len = max_length
        self.input_ids, self.attention_mask,self.token_type_ids,self.label = self.get_input(df)

    def __len__(self):
        return len(self.label)
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens_seq)
        input_ids += padding   
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len)
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids

    def get_input(self,df):
        tweet = df['text'].values
        label = df['label'].values
        tokens_seq = list(map(self.tokenizer.tokenize,tweet))
        result = list(map(self.trunate_and_pad, tokens_seq))
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]

        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(label).type(torch.long)
               )

    def __getitem__(self, item):
        return self.input_ids[item], self.attention_mask[item],self.token_type_ids[item],self.label[item]


## Model class

In [10]:
def model_init(model_name,lr,weight_decay,batch_size,warmup,data):
  model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels = 2)
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
          {
                  'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                  'weight_decay':weight_decay
          },
          {
                  'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                  'weight_decay':0.0
          }
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr, betas = (0.9,0.98),eps = 1e-6)
  total_steps = int(n_epochs * (len(data["train"]["text"]) / batch_size))
  warmup_steps = int(warmup * total_steps)
  # print("Total # training steps: {}, # warmup steps: {}".format(total_steps, warmup_steps))
  # This scheduler is not available in PyTorch, but it is in HuggingFace => it is super important to use it with Transformer-based models
  # Same with the AdamW optimizer: not in PyTorch, but import it from HuggingFace instead 
  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps = warmup_steps, 
      num_training_steps = total_steps)
  return model,optimizer,scheduler


## Putting data, datasets, loaders and models together

3 classes make it simple to load and access a pre-trained language model hosted on HuggingFace's platform:

*   **AutoConfig**: load the model configuration (dictionary of parameters) given its name
*   **AutoModel**: load the model given its name
*   **AutoTokenizer**: load the associated tokenizer given the model name

In [11]:
# dataset specifications
model_name = 'roberta-base'
max_length = 50
batch_size = 32


In [12]:
# Tip: specify a cache directory through the cache_dir argument to avoid re-downloading from HuggingFace's cloud the tokenizer, pre-trained model each time 
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir = "."
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [13]:
# datasets
train_dataset = SST2Dataset(tokenizer, data_train, max_length)
val_dataset = SST2Dataset(tokenizer, data_val, max_length)
test_dataset = SST2Dataset(tokenizer, data_test, max_length)

In [14]:
# data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

Load Pre-trained Roberta

In [15]:
# Build the pre-trained model encoder
pretrained_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels = 2) # remember AutoModel
n_params = sum(p.numel() for p in pretrained_model.parameters() if p.requires_grad)
print("\nThe pre-trained model has {} trainable parameters".format(n_params))

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi


The pre-trained model has 124647170 trainable parameters


## Optimization

In [16]:
# Hyper-parameters
n_epochs = 10
lr = 3e-5
warmup = 0.06
max_gradient_norm = 10
weight_decay = 0.1
# for param in model.base_model.parameters():
#     param.requires_grad = False
# model.to(device)
patience = 3

## Training loop

In [17]:
def set_seed(seed_value = 0):
    """Set seed for reproducibility.
    """
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    
def train(train_loader, model, optimizer, scheduler):
    model.train()
    train_losses = []
    correct_preds = 0
    tqdm_loader = tqdm(train_loader)
    for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in enumerate (tqdm_loader):
        model.zero_grad()
        

        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        segments = segments.to(device)
        labels = labels.to(device)

        loss,logits = model(input_ids, attention_mask,segments,labels = labels)[:2] # forward pass (batch_size, 1)
        probs = nn.functional.softmax(logits,dim = -1)
        _,out_classes = probs.max(dim =1)
        correct = (out_classes == labels).sum()
        correct_preds += correct.item()
        


        # outs = outs[:,:] # (batch_size)
        # print (f'train outs shape : {outs.shape}')
        # print (f'train labels shape : {labels.shape}')

        # loss = loss_fct(outs.view(-1,d_out), labels.view(-1))
        train_losses.append(loss.item())

        loss.backward() # backward pass
        ## clip grad
        nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step() # weights update
        
        scheduler.step()
        optimizer.zero_grad()
        
    train_loss = np.mean(train_losses)
    epoch_accuracy = correct_preds /len(train_loader.dataset)

    return train_loss,epoch_accuracy

In [18]:
def eval(loader, model):
    model.eval()
    val_losses = []
    val_probs = []
    acc_correct = 0
    iter = 0
    log_interval = 100
    sigmoid = nn.Sigmoid()

    val_labels = []
    # threshold = torch.tensor([0.5]).to(device)
    # loader = tqdm(loader)
    for (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in loader:
        
        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        segments = segments.to(device)
        # print (attention_mask.shape,labels.shape)

        with torch.no_grad():
            valid_loss,valid_logits = model(input_ids, attention_mask,segments,labels = labels)[:2]
            valid_probs = nn.functional.softmax(valid_logits,dim = -1)
            _,out_classes = valid_probs.max(dim =1)
            correct = (out_classes == labels).sum()
            acc_correct += correct.item()
            val_probs.extend(valid_probs[:,1].cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_losses.append(valid_loss.item())
    val_loss = np.mean(val_losses)
    val_probs = np.array(val_probs)
    val_labels = np.array(val_labels)
    valid_accuracy = acc_correct/(len(loader.dataset))

    
    fpr, tpr, threshold = roc_curve(val_labels, val_probs)
    val_auc = 100 * auc(fpr, tpr)

    return val_loss, val_auc, val_probs, val_labels,valid_accuracy

In [19]:
# sst2_result_folder = os.path.join('results','sst2')
# if not os.path.exists(sst2_result_folder):
#   os.makedirs(sst2_result_folder)
evaluations = torch.zeros((5,2))
for seed_val in range(5):
  set_seed(seed_value = seed_val)
  model,optimizer,scheduler = model_init(model_name,lr,weight_decay,batch_size,warmup,data)
  model.to(device)
  val_aucs = []
  val_accs = []
  best_score = 0.
  patience_counter = 0
  val_loss, val_auc, _, _, val_acc= eval(val_loader, model)
  val_aucs.append(val_auc)
  val_accs.append(val_acc)
  print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}".format(val_loss, val_auc,val_acc*100))
  for epoch in range(1, n_epochs + 1):
      
      print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

      # Training round
      train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler)

      # Validation round
      val_loss, val_auc, _, _,val_acc= eval(val_loader, model)
      val_aucs.append(val_auc)

      print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
    
      print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, (best ACC: {:.4f} at epoch {})"
      .format(val_loss, val_auc,val_acc*100,np.max(np.array(val_acc)),np.argmax(np.array(val_acc*100))))
      # scheduler.step(val_acc)


      print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
      if val_auc == np.max(np.array(val_aucs)):
          print("!! New best val AUC !!")
      if val_acc == np.max(np.array(val_acc)):
          print("!! New best val ACC !!")
  _, test_auc, test_probs, test_labels,test_acc = eval(test_loader, model) # val_loss, val_auc, val_probs, val_labels,val_acc
  print("Test AUC: {:.2f} seed : {}".format(test_auc,seed_val))
  print ('Test ACC : {:.4f} seed : {}'.format(test_acc*100,seed_val))
  evaluations[seed_val,0] = test_auc
  evaluations[seed_val,1] = test_acc
mean_auc = torch.mean(evaluations[:,0])
mean_acc = torch.mean(evaluations[:,1])
print ("Mean test AUC over 5 seeds: {:.2f}".format(mean_auc))
print ("Mean test ACC over 5 seeds: {:.2f}".format(mean_acc*100))

# save results of 5 seed
# torch.save(evaluations,f'{sst2_result_folder}/baseline.pt')
      # if val_acc < best_score:
      #   patience_counter += 1
      # else:
      #   best_score = val_acc
      #   patience_counter = 0
      
      # if patience_counter >= patience:
      #   print ('Early stopping, patience reached, accuracy of {:.4f} achieve, total epochs ran: {}'.format(val_acc*100,epoch))
      #   print (f'model state dict: {model.state_dict().keys()}')
      #   break
    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi


Epoch 0 (before training) val loss: 0.6931, AUC: 55.61, ACC: 50.9174

New epoch, epoch 1 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.4443, accuracy 76.6763
Validation: loss: 0.2477, AUC: 97.07, ACC : 91.7431, (best ACC: 0.9174 at epoch 0)
lr : 2.8701426463354646e-05
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 2 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

...

Training: loss: 0.0113, accuracy 99.6965
Validation: loss: 0.5042, AUC: 97.22, ACC : 92.2018, (best ACC: 0.9220 at epoch 0)
lr : 6.286276438760452e-06
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.0071, accuracy 99.7977
Validation: loss: 0.5778, AUC: 96.86, ACC : 92.0872, (best ACC: 0.9209 at epoch 0)
lr : 3.08411214953271e-06
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/217 [00:00<?, ?it/s]

Training: loss: 0.0039, accuracy 99.8988
Validation: loss: 0.6166, AUC: 97.12, ACC : 91.7431, (best ACC: 0.9174 at epoch 0)
lr : 0.0
!! New best val ACC !!
Test AUC: 98.62 seed : 4
Test ACC : 94.3987 seed : 4
Mean test AUC over 5 seeds: 98.75
Mean test ACC over 5 seeds: 0.94


In [21]:
print ('Test mean ACC : {:.2f}'.format(mean_acc*100))

Test mean ACC : 94.43


In [ ]:
# Qualitative test samples
n_check = 10
label_name = dict({'1.0':'positive','0.0':'negative'})
for i in range(n_check):
    tweet = data["test"]["text"][i]
    test_label = test_labels[i]  
    prob = test_probs[i]
    print("\n" + "*"*20, "Tweet {}:".format(i+1))
    print("\n".join(sent_tokenize(tweet))) # print 1 sentence per line for reading easiness
    print("*"*10, "Label: {}".format(label_name[str(float(test_label))]))
    print ("*"*10, "Label: {}".format(test_label))
    print("*"*10, "Model prediction: {:.4f}".format(prob))